In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import torchvision
from torchvision import transforms
from IPython import display
from datetime import datetime
import sklearn
from sklearn.model_selection import TimeSeriesSplit
#import pdb
#import json
#import sys
import random
from time import time
from pathlib import Path
from typing import Optional, Protocol, Union, Dict, Iterator, Tuple
import torch.nn.functional as F
from sklearn.metrics import auc

# Parameters

In [2]:
num_generated_features = 6

random_seed = 0

num_epochs = 100
batch_size = 256
lr = 0.001

latent_dim = 15
hidden_dim = 100
seq_length = 30
seq_stride = 10

# Load data

In [3]:
def kdd99(seq_length, seq_step, num_signals):
    train = np.load('./data/kdd99_train.npy')
    print('load kdd99_train from .npy')
    m, n = train.shape  # m=562387, n=35
    # normalization
    for i in range(n - 1):
        # print('i=', i)
        A = max(train[:, i])
        # print('A=', A)
        if A != 0:
            train[:, i] /= max(train[:, i])
            # scale from -1 to 1
            train[:, i] = 2 * train[:, i] - 1
        else:
            train[:, i] = train[:, i]

    samples = train[:, 0:n - 1]
    labels = train[:, n - 1]  # the last colummn is label
    #############################
    ############################
    # -- apply PCA dimension reduction for multi-variate GAN-AD -- #
    from sklearn.decomposition import PCA
    X_n = samples
    ####################################
    ###################################
    # -- the best PC dimension is chosen pc=6 -- #
    n_components = num_signals
    pca = PCA(n_components, svd_solver='full')
    pca.fit(X_n)
    ex_var = pca.explained_variance_ratio_
    pc = pca.components_
    # projected values on the principal component
    T_n = np.matmul(X_n, pc.transpose(1, 0))
    samples = T_n
    # # only for one-dimensional
    # samples = T_n.reshape([samples.shape[0], ])
    ###########################################
    ###########################################
    num_samples = (samples.shape[0] - seq_length) // seq_step
    aa = np.empty([num_samples, seq_length, num_signals])
    bb = np.empty([num_samples, seq_length, 1])

    for j in range(num_samples):
        bb[j, :, :] = np.reshape(labels[(j * seq_step):(j * seq_step + seq_length)], [-1, 1])
        for i in range(num_signals):
            aa[j, :, i] = samples[(j * seq_step):(j * seq_step + seq_length), i]

    samples = aa
    labels = bb

    return samples, labels

def kdd99_test(seq_length, seq_step, num_signals):
    test = np.load('./data/kdd99_test.npy')
    print('load kdd99_test from .npy')

    m, n = test.shape  # m1=494021, n1=35

    for i in range(n - 1):
        B = max(test[:, i])
        if B != 0:
            test[:, i] /= max(test[:, i])
            # scale from -1 to 1
            test[:, i] = 2 * test[:, i] - 1
        else:
            test[:, i] = test[:, i]

    samples = test[:, 0:n - 1]
    labels = test[:, n - 1]
    idx = np.asarray(list(range(0, m)))  # record the idx of each point
    #############################
    ############################
    # -- apply PCA dimension reduction for multi-variate GAN-AD -- #
    from sklearn.decomposition import PCA
    X_a = samples
    ####################################
    ###################################
    # -- the best PC dimension is chosen pc=6 -- #
    n_components = num_signals
    pca_a = PCA(n_components, svd_solver='full')
    pca_a.fit(X_a)
    pc_a = pca_a.components_
    # projected values on the principal component
    T_a = np.matmul(X_a, pc_a.transpose(1, 0))
    samples = T_a
    # # only for one-dimensional
    # samples = T_a.reshape([samples.shape[0], ])
    ###########################################
    ###########################################
    num_samples_t = (samples.shape[0] - seq_length) // seq_step
    aa = np.empty([num_samples_t, seq_length, num_signals])
    bb = np.empty([num_samples_t, seq_length, 1])
    bbb = np.empty([num_samples_t, seq_length, 1])

    for j in range(num_samples_t):
        bb[j, :, :] = np.reshape(labels[(j * seq_step):(j * seq_step + seq_length)], [-1, 1])
        bbb[j, :, :] = np.reshape(idx[(j * seq_step):(j * seq_step + seq_length)], [-1, 1])
        for i in range(num_signals):
            aa[j, :, i] = samples[(j * seq_step):(j * seq_step + seq_length), i]

    samples = aa
    labels = bb
    index = bbb

    return samples, labels, index

In [4]:
train_samples, train_labels = kdd99(seq_length, seq_stride, num_generated_features)
test_samples, test_labels, index = kdd99_test(seq_length, seq_stride, num_generated_features)

load kdd99_train from .npy
load kdd99_test from .npy


In [5]:
train_x = torch.Tensor(train_samples) 
train_y = torch.Tensor(train_labels) 
train_data = torch.utils.data.TensorDataset(train_x,train_y)
train_dl = torch.utils.data.DataLoader(train_data, shuffle=False, batch_size=batch_size)

test_x = torch.Tensor(test_samples) 
test_y = torch.Tensor(test_labels) 
test_data = torch.utils.data.TensorDataset(test_x,test_y)
test_dl = torch.utils.data.DataLoader(test_data, shuffle=False, batch_size=batch_size)

# Complementary functions

In [6]:
def set_seed(seed: int = 0) -> None:
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [7]:
# --- sample Z from latent space --- #
def sample_Z(batch_size, seq_length, latent_dim, use_time=False, use_noisy_time=False):
    sample = np.float32(np.random.normal(size=[batch_size, seq_length, latent_dim]))
    if use_time:
        print('WARNING: use_time has different semantics')
        sample[:, :, 0] = np.linspace(0, 1.0 / seq_length, num=seq_length)
    return torch.Tensor(sample)

# Model

In [8]:
class Generator(nn.Module):
    def __init__(self,latent_space_dim: int,hidden_units: int,output_dim: int,n_lstm_layers: int = 2) -> None:
        super().__init__()
        self.latent_space_dim = latent_space_dim
        self.hidden_units = hidden_units
        self.n_lstm_layers = n_lstm_layers
        self.output_dim = output_dim

        self.lstm = nn.LSTM(input_size=self.latent_space_dim,
                            hidden_size=self.hidden_units,
                            num_layers=self.n_lstm_layers,
                            batch_first=True,
                            dropout=.1)

        self.linear = nn.Linear(in_features=self.hidden_units,
                                out_features=self.output_dim)
        
        nn.init.trunc_normal_(self.linear.bias)
        nn.init.trunc_normal_(self.linear.weight)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        rnn_output, _ = self.lstm(x)
        return self.linear(rnn_output)

    def save(self, fpath: Union[Path, str]) -> None:
        chkp = {
            "config": {
                "latent_space_dim": self.latent_space_dim,
                "hidden_units": self.hidden_units,
                "n_lstm_layers": self.n_lstm_layers,
                "output_dim": self.output_dim
            },
            "weights": self.state_dict(),
        }
        torch.save(chkp, fpath)
    
    @classmethod
    def from_pretrained(
            cls,
            fpath: Union[Path, str],
            map_location: Optional[torch.device] = None) -> "Generator":
        chkp = torch.load(fpath, map_location=map_location)
        model = cls(**chkp.pop("config"))
        model.load_state_dict(chkp.pop("weights"))
        model.eval()
        return model

In [9]:
class Discriminator(nn.Module):
    def __init__(self,input_dim, hidden_units, n_lstm_layers: int = 2,add_batch_mean: bool = True) -> None:
        super().__init__()
        self.add_batch_mean = add_batch_mean
        self.hidden_units = hidden_units
        self.input_dim = input_dim
        self.n_lstm_layers = n_lstm_layers

        extra_features = self.hidden_units if self.add_batch_mean else 0
        self.lstm = nn.LSTM(input_size=self.input_dim,
                            hidden_size=self.hidden_units + extra_features,
                            num_layers=self.n_lstm_layers,
                            batch_first=True,
                            dropout=.1)

        self.linear = nn.Linear(in_features=self.hidden_units + extra_features,
                                out_features=1)
        nn.init.trunc_normal_(self.linear.bias)
        nn.init.trunc_normal_(self.linear.weight)

        self.activation = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.add_batch_mean:
            bs = x.size(0)
            batch_mean = x.mean(0, keepdim=True).repeat(bs, 1, 1)
            x = torch.cat([x, batch_mean], dim=-1)

        rnn_output, _ = self.lstm(x)
        return self.activation(self.linear(rnn_output))

    def save(self, fpath: Union[Path, str]) -> None:
        chkp = {
            "config": {
                "add_batch_mean": self.add_batch_mean,
                "hidden_units": self.hidden_units,
                "input_dim": self.input_dim,
                "n_lstm_layers": self.n_lstm_layers
            },
            "weights": self.state_dict(),
        }
        torch.save(chkp, fpath)
        
    @classmethod
    def from_pretrained(
            cls,
            fpath: Union[Path, str],
            map_location: Optional[torch.device] = None) -> "Discriminator":
        chkp = torch.load(fpath, map_location=map_location)
        model = cls(**chkp.pop("config"))
        model.load_state_dict(chkp.pop("weights"))
        model.eval()
        return model

# Initialization

In [10]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
generator = Generator(
    latent_space_dim=latent_dim,
    hidden_units=hidden_dim,
    output_dim=num_generated_features)
generator.to(DEVICE)

Generator(
  (lstm): LSTM(15, 100, num_layers=2, batch_first=True, dropout=0.1)
  (linear): Linear(in_features=100, out_features=6, bias=True)
)

In [12]:
discriminator = Discriminator(input_dim=num_generated_features,
    hidden_units=hidden_dim,
    add_batch_mean=False)
discriminator.to(DEVICE)

Discriminator(
  (lstm): LSTM(6, 100, num_layers=2, batch_first=True, dropout=0.1)
  (linear): Linear(in_features=100, out_features=1, bias=True)
  (activation): Sigmoid()
)

# Loss and Optimizer

In [13]:
def loss_function(inputs, targets):
    return nn.BCELoss()(inputs, targets)

In [14]:
discriminator_optim = torch.optim.Adam(discriminator.parameters(), lr=lr)
generator_optim = torch.optim.Adam(generator.parameters(), lr=lr)

# Evaluation

In [15]:
@torch.no_grad()
def evaluate(G, D, loss_fn, real_dl, normal_label: int = 0, anomaly_label: int = 1) -> Dict[str, float]: 
    metric_accum = {
        "D_loss": 0,
        "G_acc": 0,
        "D_acc": 0
    }
    batch_count = 0
    for X, Y in real_dl:
        bs = X.size(0)
        
        # Samples
        real_samples = X.to(DEVICE)
        latent_samples = sample_Z(bs, seq_length, latent_dim).to(DEVICE)
        fake_samples = G(latent_samples)

        # Labels
        real_labels = Y.to(DEVICE)
        fake_labels = torch.zeros(bs, seq_length, 1).to(DEVICE)
        all_labels = torch.cat([real_labels, fake_labels])

        # Try to classify the real and generated samples
        real_d = D(real_samples)
        fake_d = D(fake_samples.detach())
        all_d = torch.cat([real_d, fake_d]).to(DEVICE)

        # Discriminator tries to identify the true nature of each sample (anomaly or normal)
        d_real_loss = loss_fn(real_d.view(-1), real_labels.view(-1))
        d_fake_loss = loss_fn(fake_d.view(-1), fake_labels.view(-1))
        d_loss = d_real_loss + d_fake_loss

        discriminator_acc = ((all_d > .5) == all_labels).float()
        discriminator_acc = discriminator_acc.sum().div(bs*seq_length)

        generator_acc = ((fake_d > .5) == real_labels).float()
        generator_acc = generator_acc.sum().div(bs*seq_length)

        metric_accum["D_loss"] += d_loss.item()
        metric_accum["D_acc"] += discriminator_acc.item()
        metric_accum["G_acc"] += generator_acc.item()
        batch_count += 1
    for key in metric_accum.keys():
        metric_accum[key] = metric_accum[key]/batch_count
    print("Evaluation metrics:", metric_accum)
    return metric_accum

# Train

In [16]:
def train_epoch(G, D, loss_fn, real_dl, G_optimizer, D_optimizer, normal_label: int = 0, anomaly_label: int = 1, epoch: int = 0, log_every: int = 30) -> None:
    G.train()
    D.train()
    metric_accum = {
    "generator_loss": 0,
    "discriminator_loss_real": 0,
    "discriminator_loss_fake": 0,
    }
    batch_count = 0
    for i, (X,Y) in enumerate(real_dl):
        bs = X.size(0)
        
        # Samples
        real_samples = X.to(DEVICE)
        latent_samples = sample_Z(bs, seq_length, latent_dim).to(DEVICE)
        fake_samples = G(latent_samples)

        # Labels
        real_labels = Y.to(DEVICE)
        fake_labels = torch.full((bs, seq_length, 1), anomaly_label).float().to(DEVICE)
        all_labels = torch.cat([real_labels, fake_labels])
        
        # Discriminator update
        D_optimizer.zero_grad()
        real_d = D(real_samples)
        fake_d = D(fake_samples.detach())
        all_d = torch.cat([real_d, fake_d])
        
        d_loss_real = loss_fn(real_d.view(-1), real_labels.view(-1))
        d_loss_fake = loss_fn(fake_d.view(-1), fake_labels.view(-1))
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        
        D_optimizer.step()

        #Genertor update
        G_optimizer.zero_grad()
        fake_d = D(fake_samples)
        g_loss = loss_fn(fake_d.view(-1), real_labels.view(-1))
        g_loss.backward()
        G_optimizer.step()
        
        # Save metrics
        metric_accum['generator_loss'] += g_loss.item()
        metric_accum['discriminator_loss_real'] += d_loss_real.item()
        metric_accum['discriminator_loss_fake'] += d_loss_fake.item()
        batch_count += 1
    D.zero_grad()
    G.zero_grad()
    out = 'G_loss: ' + str(metric_accum['generator_loss']/batch_count) + ', ' + 'D_loss_real: ' + str(metric_accum['discriminator_loss_real']/batch_count) + ', ' + 'D_loss_fake: ' + str(metric_accum['discriminator_loss_fake']/batch_count)
    print('Epoch ' + str(epoch) + 'training:')
    print(out)

In [17]:
def train(batch_size, seq_length, latent_dim, train_dl, test_dl, D, G, D_optim, G_optim, loss_fn,
          model_dir: Path = Path("models/madgan")) -> None:
    set_seed(random_seed)
    model_dir.mkdir(parents=True, exist_ok=True)

    for epoch in range(num_epochs):
        train_epoch(G, D, loss_fn, train_dl, generator_optim, discriminator_optim, normal_label=0, anomaly_label=1, epoch=epoch)
        evaluate(G, D, loss_fn, test_dl, normal_label=0, anomaly_label=1)

        G.save(model_dir / f"generator_{epoch}.pt")
        D.save(model_dir / f"discriminator_{epoch}.pt")

In [18]:
train(batch_size, seq_length, latent_dim, train_dl, test_dl, discriminator, generator, discriminator_optim, generator_optim, loss_function)

Epoch 0training:
G_loss: 2.6426471847024833, D_loss_real: 0.6577470123090527, D_loss_fake: 0.680391707948663
Evaluation metrics: {'D_loss': 4.524801220918566, 'G_acc': 0.7562280367538718, 'D_acc': 0.34170340013164313}
Epoch 1training:
G_loss: 2.8840485973791643, D_loss_real: 0.46929060225798325, D_loss_fake: 0.3836389892649921
Evaluation metrics: {'D_loss': 4.165808588729621, 'G_acc': 0.7902210044911026, 'D_acc': 0.3741761211672595}
Epoch 2training:
G_loss: 2.1145877041599968, D_loss_real: 0.5924394269219854, D_loss_fake: 0.46860516633499755
Evaluation metrics: {'D_loss': 1.9284354188899302, 'G_acc': 0.7308149179491972, 'D_acc': 0.8842415930075966}
Epoch 3training:
G_loss: 1.9815886093811554, D_loss_real: 0.48248353075574746, D_loss_fake: 0.4688820813528516
Evaluation metrics: {'D_loss': 6.067451795765773, 'G_acc': 0.7175702901238604, 'D_acc': 0.2713425062944234}
Epoch 4training:
G_loss: 2.119596509229053, D_loss_real: 0.6370347268879414, D_loss_fake: 0.4514860008927909
Evaluation metr

Evaluation metrics: {'D_loss': 2.0395104100667134, 'G_acc': 0.7402330250551663, 'D_acc': 0.9481771464406518}
Epoch 38training:
G_loss: 1.9732234179973602, D_loss_real: 0.404508914231238, D_loss_fake: 0.30722422152757645
Evaluation metrics: {'D_loss': 3.452962250289522, 'G_acc': 0.7603456573705599, 'D_acc': 0.7396899185341257}
Epoch 39training:
G_loss: 1.9203370289369064, D_loss_real: 0.3838494819165631, D_loss_fake: 0.32186033007773485
Evaluation metrics: {'D_loss': 3.8649214926161295, 'G_acc': 0.7375693744328355, 'D_acc': 0.369807144559418}
Epoch 40training:
G_loss: 1.6941676684401252, D_loss_real: 0.45999550175937737, D_loss_fake: 0.4033679587597197
Evaluation metrics: {'D_loss': 2.568090862560766, 'G_acc': 0.7363751988105206, 'D_acc': 0.9294062838576}
Epoch 41training:
G_loss: 2.030965313586322, D_loss_real: 0.3925564662976698, D_loss_fake: 0.3334751819345084
Evaluation metrics: {'D_loss': 2.611526700499144, 'G_acc': 0.7014193222158314, 'D_acc': 1.0130544321061417}
Epoch 42training:

Epoch 75training:
G_loss: 1.6488267909396779, D_loss_real: 0.45174915978727354, D_loss_fake: 0.3169500908391042
Evaluation metrics: {'D_loss': 2.3962712275549536, 'G_acc': 0.7854230499638177, 'D_acc': 0.9692654160425608}
Epoch 76training:
G_loss: 1.7350665520537982, D_loss_real: 0.43364922074939716, D_loss_fake: 0.27788718437606635
Evaluation metrics: {'D_loss': 2.4108618812857516, 'G_acc': 0.7875066390791859, 'D_acc': 0.9656543154076437}
Epoch 77training:
G_loss: 1.5996628587896173, D_loss_real: 0.4956960573580793, D_loss_fake: 0.39395454519174317
Evaluation metrics: {'D_loss': 2.0266853447405166, 'G_acc': 0.7863158862362254, 'D_acc': 0.9671651127366977}
Epoch 78training:
G_loss: 1.5747747402299535, D_loss_real: 0.4772223069417206, D_loss_fake: 0.4181711177256974
Evaluation metrics: {'D_loss': 1.9424785536187918, 'G_acc': 0.7275563579226405, 'D_acc': 1.0637188411747236}
Epoch 79training:
G_loss: 1.498180402950807, D_loss_real: 0.4903134814124893, D_loss_fake: 0.41442872773517264
Evalu

# Anomaly Detector

In [19]:
class AnomalyDetector(object):

    def __init__(self,
                 *,
                 discriminator: nn.Module,
                 generator: nn.Module,
                 latent_space_dim: int,
                 res_weight: float = .2,
                 anomaly_threshold: float = 1.0) -> None:
        self.discriminator = discriminator.to('cpu')
        self.generator = generator.to('cpu')
        self.threshold = anomaly_threshold
        self.latent_space_dim = latent_space_dim
        self.res_weight = res_weight

    def predict(self, tensor: torch.Tensor) -> torch.Tensor:
        return (self.predict_proba(tensor) > self.threshold).int()

    def predict_proba(self, tensor: torch.Tensor) -> torch.Tensor:
        discriminator_score = self.compute_anomaly_score(tensor)
        discriminator_score *= 1 - self.res_weight
        reconstruction_loss = self.compute_reconstruction_loss(tensor)
        reconstruction_loss *= self.res_weight
        return discriminator_score #+ reconstruction_loss

    def compute_anomaly_score(self, tensor: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            discriminator_score = self.discriminator(tensor)
        return discriminator_score

    def compute_reconstruction_loss(self,
                                    tensor: torch.Tensor) -> torch.Tensor:
        best_reconstruct = self._generate_best_reconstruction(tensor)
        return (best_reconstruct - tensor).abs().sum(dim=(1, 2))

    def _generate_best_reconstruction(self, tensor: torch.Tensor) -> None:
        # The goal of this function is to find the corresponding latent space for the given
        # input and then generate the best possible reconstruction.
        max_iters = 10

        Z = torch.empty(
            (tensor.size(0), tensor.size(1), self.latent_space_dim),
            requires_grad=True)
        nn.init.normal_(Z, std=0.05)

        optimizer = torch.optim.RMSprop(params=[Z], lr=0.1)
        loss_fn = nn.MSELoss(reduction="none")
        normalized_target = F.normalize(tensor, dim=1, p=2)

        for _ in range(max_iters):
            optimizer.zero_grad()
            generated_samples = self.generator(Z)
            normalized_input = F.normalize(generated_samples, dim=1, p=2)
            reconstruction_error = loss_fn(normalized_input,
                                           normalized_target).sum(dim=(0, 1, 2))
            reconstruction_error.backward()
            optimizer.step()

        with torch.no_grad():
            best_reconstruct = self.generator(Z)
        return best_reconstruct

# EMMV Eval

In [20]:
def excess_mass(t, t_max, volume_support, s_unif, s_X, n_generated):
    EM_t = np.zeros(t.shape[0])
    n_samples = s_X.shape[0]
    s_X_unique = np.unique(s_X)
    EM_t[0] = 1.

    for u in s_X_unique:
        EM_t = np.maximum(EM_t, 1. / n_samples * (s_X > u).sum() -
                        t * (s_unif > u).sum() / n_generated
                        * volume_support)
    amax = np.argmax(EM_t <= t_max) + 1
    if amax == 1:
        amax = -1 # failed to achieve t_max
    AUC = auc(t[:amax], EM_t[:amax])
    return AUC, EM_t, amax

In [21]:
def mass_volume(axis_alpha, volume_support, s_unif, s_X, n_generated):
    n_samples = s_X.shape[0]
    s_X_argsort = s_X.argsort()
    mass = 0
    cpt = 0
    u = s_X[s_X_argsort[-1]]
    mv = np.zeros(axis_alpha.shape[0])
    for i in range(axis_alpha.shape[0]):
        while mass < axis_alpha[i]:
            cpt += 1
            u = s_X[s_X_argsort[-cpt]]
            mass = 1. / n_samples * cpt  # sum(s_X > u)
        mv[i] = float((s_unif >= float(u)).sum()) / n_generated * volume_support
    return auc(axis_alpha, mv), mv

In [22]:
def emmv_scores(trained_model, df, scoring_func=None, n_generated=10000, alpha_min=0.9, alpha_max=0.999, t_max=0.9):
    """Get Excess-Mass (EM) and Mass Volume (MV) scores for unsupervised ML AD models.
    :param trained_model: Trained ML model with a 'decision_function' method 
    :param df: Pandas dataframe of features (X matrix)
    :param scoring_func: Anomaly scoring function (callable)
    :param n_generated: , defaults to 10000
    :param alpha_min: Min value for alpha axis, defaults to 0.9
    :param alpha_max: Max value for alpha axis, defaults to 0.999
    :param t_max: Min EM value required, defaults to 0.9
    :return: A dictionary of two scores ('em' and 'mv')
    """

    if scoring_func is None:
        scoring_func = lambda model, df: model.decision_function(df)

    # Get limits and volume support.
    lim_inf = df.min(axis=0) 
    lim_sup = df.max(axis=0)
    offset = 1e-60 # to prevent division by 0
    try:
        volume_support = (lim_sup - lim_inf).prod() + offset
    except AttributeError as e: # i.e Pandas Series
        volume_support = (lim_sup - lim_inf) + offset

    # Determine EM and MV parameters
    t = np.arange(0, 100 / volume_support, 0.01 / volume_support)
    axis_alpha = np.arange(alpha_min, alpha_max, 0.0001)

    try:
        unif = np.random.uniform(lim_inf, lim_sup, size=(n_generated, df.shape[1]))
    except IndexError as e: # i.e. 1D data
        unif = np.random.uniform(lim_inf, lim_sup, size=(n_generated))
        
    # Get anomaly scores
    anomaly_score = scoring_func(trained_model, df)#.reshape(1, -1)[0]
    s_unif = scoring_func(trained_model, unif)
    
    # Get EM and MV scores
    AUC_em, em, amax = excess_mass(t, t_max, volume_support, s_unif, anomaly_score, n_generated)
    AUC_mv, mv = mass_volume(axis_alpha, volume_support, s_unif, anomaly_score, n_generated)
    # Return a dataframe containing EMMV information
    scores = {
        'em': np.mean(em),
        'mv': np.mean(mv),
    }
    return scores

In [23]:
def torch_emmv_scores(trained_model, x, scoring_func=None, n_generated=10000, alpha_min=0.9, alpha_max=0.999, t_max=0.9):
    
    # Get limits and volume support.
    lim_inf = torch.min(x.view(-1,6), dim=0)[0]
    lim_sup = torch.max(x.view(-1,6), dim=0)[0]
    offset = 1e-60 # to prevent division by 0

    # Volume support
    volume_support = torch.prod(lim_sup - lim_inf).item() + offset

    # Determine EM and MV parameters
    t = np.arange(0, 100 / volume_support, 0.01 / volume_support)
    axis_alpha = np.arange(alpha_min, alpha_max, 0.0001)

    unif = torch.rand(n_generated, x.size(1), x.size(2))
    m = lim_sup - lim_inf
    unif = unif * m
    unif = unif + lim_inf

    # Get anomaly scores
    anomaly_score = scoring_func(trained_model, x).view(-1,1).numpy()
    s_unif = scoring_func(trained_model, unif).view(-1,1).numpy()
    
    # Get EM and MV scores
    AUC_em, em, amax = excess_mass(t, t_max, volume_support, s_unif, anomaly_score, n_generated)
    AUC_mv, mv = mass_volume(axis_alpha, volume_support, s_unif, anomaly_score, n_generated)

    # Return a dataframe containing EMMV information
    scores = {
        'em': np.mean(em),
        'mv': np.mean(mv),
    }
    return scores

In [24]:
def scoring_function(model, data):
    x = torch.tensor(data, dtype=torch.float32).unsqueeze(dim=0)
    out = model.predict(x).squeeze()
    return out.numpy()

In [25]:
def torch_scoring_function(model, data):
    return model.predict(data)

In [26]:
ad = AnomalyDetector(discriminator=discriminator, generator=generator, latent_space_dim=latent_dim, anomaly_threshold=0.5)
total_em = total_mv = 0
for X, Y in test_dl:
    prediction = ad.predict(X)
    acc = (prediction == Y).float()
    acc = acc.sum().div(batch_size)/30
    print(acc)
    scores = torch_emmv_scores(ad,X,torch_scoring_function)
    print(scores)
    total_em += scores['em']
    total_mv += scores['mv']
print(total_em/len(test_dl),total_mv/len(test_dl))

tensor(0.9992)
{'em': 0.0001, 'mv': 14253.036804199219}
tensor(0.9992)
{'em': 0.0001, 'mv': 11815.361022949219}
tensor(0.9992)
{'em': 0.0001, 'mv': 8953.750305175781}
tensor(0.1228)
{'em': 0.00010218957500000001, 'mv': 7061.455993652344}
tensor(0.5569)
{'em': 0.00010404320000000001, 'mv': 1353.2530444702147}
tensor(0.9995)
{'em': 0.0001, 'mv': 6019.761657714844}
tensor(0.9918)
{'em': 0.0001, 'mv': 30586.137084960938}
tensor(0.9613)
{'em': 0.0001, 'mv': 2632.2827911376953}
tensor(0.9732)
{'em': 0.0001, 'mv': 61709.28955078125}
tensor(1.)
{'em': 0.0001, 'mv': 1064.2021179199219}
tensor(0.9544)
{'em': 0.00010443978333333332, 'mv': 30360.598754882812}
tensor(0.9997)
{'em': 0.0001, 'mv': 1038.9903259277344}
tensor(0.9996)
{'em': 0.0001, 'mv': 9793.584594726562}
tensor(1.)
{'em': 0.0001, 'mv': 2186.026840209961}
tensor(1.)
{'em': 0.0001, 'mv': 1724.6946716308594}
tensor(0.6082)
{'em': 9.999000099990002e-05, 'mv': 47604.17724609375}
tensor(0.7780)
{'em': 0.00010079664166666666, 'mv': 129678.6

tensor(0.0806)
{'em': 0.00010918044999999999, 'mv': 1.496995821542116e-12}
tensor(0.0802)
{'em': 0.00010980251249999999, 'mv': 1.5012654588117225e-12}
tensor(0.0799)
{'em': 0.00010904100000000002, 'mv': 1.4952879666342736e-12}
tensor(0.0805)
{'em': 0.00011001648749999999, 'mv': 1.5033392826283894e-12}
tensor(0.0809)
{'em': 0.0001090536625, 'mv': 5.940434822840996e-13}
tensor(0.0803)
{'em': 0.00011073833749999999, 'mv': 1.5093777696239754e-12}
tensor(0.0828)
{'em': 0.000110238675, 'mv': 5.196860724686251e-14}
tensor(0.1051)
{'em': 0.00012519291250000003, 'mv': 992.3887781249997}
tensor(0.9138)
{'em': 0.0001, 'mv': 67595.5517578125}
tensor(0.8889)
{'em': 0.0001000086625, 'mv': 40409.293212890625}
tensor(0.4732)
{'em': 0.0001, 'mv': 137376.9140625}
tensor(0.0044)
{'em': 0.00010026608333333334, 'mv': 0.004676842654589564}
tensor(0.0051)
{'em': 0.00010061425, 'mv': 0.0052627424884121865}
tensor(0.0051)
{'em': 0.00010032464999999999, 'mv': 0.006234345637494698}
tensor(0.0043)
{'em': 0.000100